# Transfer learning

![transfer learing](https://cdn-images-1.medium.com/max/1600/0*C_2eTc2Hnu2OgXii.)

## Essential import

In [0]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Initialize some parameters

In [0]:
batch_size = 128
num_classes = 5
epochs = 5

## Train function

In [0]:
def train_model(model, train, test, num_classes): 
    x_train = train[0].reshape(train[0].shape[0], 784)
    x_test = test[0].reshape(test[0].shape[0], 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',metrics=['accuracy'])
 
    model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## Load dataset

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

## Network arch

In [0]:
# define two groups of layers: feature (dense) and classification (dense)
feature_layers = [
    Dense(512, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(512, activation='relu'),
    Dropout(0.2)
]

classification_layers = [
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

## Train Model

In [0]:
# train model for 5-digit classification [0..4]
train_model(model,(x_train_lt5, y_train_lt5),(x_test_lt5, y_test_lt5), num_classes)

30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 2s 71us/step - loss: 0.1334 - acc: 0.9570 - val_loss: 0.0493 - val_acc: 0.9856
Epoch 2/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0470 - acc: 0.9858 - val_loss: 0.0390 - val_acc: 0.9881
Epoch 3/5
30596/30596 [==============================] - 2s 51us/step - loss: 0.0343 - acc: 0.9898 - val_loss: 0.0489 - val_acc: 0.9868
Epoch 4/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0284 - acc: 0.9924 - val_loss: 0.0247 - val_acc: 0.9926
Epoch 5/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0206 - acc: 0.9941 - val_loss: 0.0268 - val_acc: 0.9926
Test score: 0.026787812183190797
Test accuracy: 0.9926055652850749


## Freeze feature layers and rebuild model

In [0]:
for l in feature_layers:
  l.trainable = False

## Transfer: train dense layers for new classification task [5..9]

In [0]:
train_model(model,(x_train_gte5, y_train_gte5),(x_test_gte5, y_test_gte5), num_classes)

29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 2s 55us/step - loss: 0.5494 - acc: 0.8322 - val_loss: 0.2580 - val_acc: 0.9159
Epoch 2/5
29404/29404 [==============================] - 1s 37us/step - loss: 0.2681 - acc: 0.9136 - val_loss: 0.1758 - val_acc: 0.9471
Epoch 3/5
29404/29404 [==============================] - 1s 37us/step - loss: 0.2240 - acc: 0.9278 - val_loss: 0.1462 - val_acc: 0.9556
Epoch 4/5
29404/29404 [==============================] - 1s 38us/step - loss: 0.2000 - acc: 0.9354 - val_loss: 0.1324 - val_acc: 0.9599
Epoch 5/5
29404/29404 [==============================] - 1s 37us/step - loss: 0.1874 - acc: 0.9385 - val_loss: 0.1246 - val_acc: 0.9609
Test score: 0.12460145065428739
Test accuracy: 0.9609133921957536


# Exercise for you

Design a image classifier for cifar10 dataset with transfer learning. 

## Tasks:
* Train the network on any 5 categories 
* Display the train and test accuracy
* Use transfer learning to train the rest of the 5 categories on the same network
* Display the train and test accuracy after tranfer learning

Use the same above code and modify to this problem.

In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()